In [43]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
import lxml

pd.set_option('display.max_colwidth', None ) #50 is the default

In [16]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Paris\AppData\Local\Temp\ipykernel_6140\794677937.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [17]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://cultural.valencia.es/es/rutas/'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)
time.sleep(5)

cookies = driver.find_element("xpath", "/html/body/div[1]/div/div/div[2]")
cookies.click()
time.sleep(2)

In [27]:
names =[]
descriptions =[]
types =[]
urls =[]
images =[]

In [28]:
soup = bs(driver.page_source,"lxml")

names_soup = soup.select("a.lnk_title")
types_soup = soup.select("div.header-metainfo")
descriptions_soup = soup.select("div.excerpt p")
urls_soup = soup.select("div.metabloque a")
images_soup = soup.select("img")

for name in names_soup:
    names.append(name.text)
for description in descriptions_soup:
    descriptions.append(description.text)
for type in types_soup:
    types.append(type.text.split()[-2:])
for url in urls_soup:
    urls.append(url['href'])

In [29]:
for image in images_soup:
    images.append(image['src'])
images = images[6:-2]

In [30]:
next_page = driver.find_element("xpath", "/html/body/div[2]/div[2]/div/article/div/div/div/div/section[1]/div/div/div[2]/div/div/div[2]/div/div/div[2]/div[3]/ul/li[2]/a")
next_page.click()
time.sleep(2)

In [33]:
soup = bs(driver.page_source,"lxml")

names_soup = soup.select("a.lnk_title")
types_soup = soup.select("div.header-metainfo")
descriptions_soup = soup.select("div.excerpt p")
urls_soup = soup.select("div.metabloque a")
images_soup = soup.select("img")

for name in names_soup:
    names.append(name.text)
for description in descriptions_soup:
    descriptions.append(description.text)
for type in types_soup:
    types.append(type.text.split()[-2:])
for url in urls_soup:
    urls.append(url['href'])

In [34]:
images_2 = []
for image_2 in images_soup:
    images_2.append(image_2['src'])
images_2 = images_2[6:-2]

In [35]:
images = images + images_2

In [38]:
dificulty = []
duration = []
start = []
end = []

for url in urls:
    driver.get(url)
    time.sleep(5)
    soup = bs(driver.page_source,"lxml")
    select_soup = soup.select("div.col-md-8 p")

    if len(select_soup[0].text) == 0:
        dificulty.append(select_soup[1].text)
    else:
        dificulty.append(None)

    if (len(select_soup) == 3) and len(select_soup[0].text) == 0:
        duration.append(None)
        start.append(None)
        end.append(None)
    else:
        duration.append(select_soup[-3].text)
        start.append(select_soup[-2].text)
        end.append(select_soup[-1].text)

In [39]:
data = {'name':names,
    'description':descriptions,
    'types':types,
    'duration':duration,
    'dificulty':dificulty,
    'start':start,
    'end':end,
    'image':images,
    'url':urls}

In [40]:
df_routes = pd.DataFrame(data)

In [41]:
df_routes.to_csv('data/routes.csv')

In [42]:
df_routes

,name,description,types,duration,dificulty,start,end,image,url
0,Árboles monumentales y singulares,Los Árboles Monumentales y Singulares reflejan...,"[bicicleta, Històrica]",2 h,baja,Plaza del Ayuntamiento,Jardines de la Glorieta y Parterre,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/testigos-...
1,Árboles: La vuelta al mundo,Ruta que contiene árboles originarios de todo ...,"[peu, Turística]",30 min,baja,Entrada Jardines del Real (C/ San Pio V),Museo de Ciencias Naturales,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/arboles-l...
2,Árboles: Los indianos,Ruta por el barrio de Ayora que finaliza al Ja...,"[peu, Turística]",1h,baja,Av. de J. J. Dómine,Palacete de Ayora,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/arboles-l...
3,Árboles: Un paseo por la alameda,Esta ruta nos traslada a este emblemático luga...,"[peu, Turística]",1h,baja,Plaza de Zaragoza,Avenida de Blasco Ibañez (Confederación hidrog...,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/arboles-u...
4,Árboles: Un viaje por el Mediterráneo,La presencia de Árboles Monumentales y Singula...,"[peu, Turística]",1 h,baja,Parque Wangari Maathai,Centro Cultural la Rambleta,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/arboles-u...
5,Calle Ruzafa – La Lonja,La ruta que transcurre entre la Calle Ruzafa y...,"[peu, Patrimonial]",6 horas,baja,Plaza de Toros,Iglesia de los Santos Juanes,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/carrer-ru...
6,Cuentos Blasco Ibañez – Ruta 1,La primera ruta de Cuentos de Blasco Ibañez pe...,"[peu, Literària]",Entre 1 hora y media y 2 horas,baja,Plaza de Toros,Casa natal de San Vicente Ferrer,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/cuentos-b...
7,Cuentos Blasco Ibañez – Ruta 2,"En este segundo itinerario, llegaremos hasta e...","[peu, Literària]",Entre 1 hora y media y 2 horas,baja,Puente de San José,La Lonja,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/cuentos-b...
8,Entre naranjos,Ateniéndonos a lo narrado en la novela Entre n...,"[peu, Literària]",Entre 1 hora y media y 2 horas,baja,Estación de Tren,Alameda,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/entre-nar...
9,Glorieta – Santa Catalina,Es un auténtico placer darse un paseo tranquil...,"[peu, Patrimonial]",4 horas,None,Santa Catalina,Glorieta,https://cultural.valencia.es/wp-content/upload...,https://cultural.valencia.es/es/ruta/glorieta-...
